### Parameters & Imports

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, LambdaCallback

img_width, img_height = 320, 240
model_path = 'models/outputs/inception_v3.h5'
frame_dir = 'datasets/UCF11/frames-backup'
features_dir = 'datasets/UCF11/features/'
nb_frame_samples = 9190
batch_size = 64
epochs = 50
nb_class = 11

Using TensorFlow backend.


### Define Model

In [2]:
base_model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
predictions = GlobalAveragePooling2D()(x)

#x = model.output
#x = Flatten()(x)
#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
#x = Dense(1024, activation='relu')(x)
#predictions = Dense(nb_class, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
#model.load_weights(model_path)

Instructions for updating:
Colocations handled automatically by placer.


### Data Generator

In [3]:
frame_datagen = ImageDataGenerator(rescale=1./255)
frame_generator = frame_datagen.flow_from_directory(
    frame_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size
)

Found 9190 images belonging to 2 classes.


### Deploy Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

saver = LambdaCallback(
    on_batch_end=lambda batch, logs: [
        print(batch)
    ])

model.predict_generator(
    frame_generator,
    (nb_frame_samples // batch_size),
    [saver],
    1
)